# そもそもどんなデータを扱うの？

ここでは，本サイトで分析対象とするデータについて簡単に紹介し，基礎分析を実施します．なお，このデータは[MADB Labでv1.0として公開されているもの](https://github.com/mediaarts-db/dataset/tree/1.0)に[こちらの前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html)を実施したものです．

## 下準備

In [34]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [35]:
# 前処理の結果，以下に分析対象ファイルが格納されていることを想定
PATH_DATA = '../../data/preprocess/out/episodes.csv'

In [36]:
df = pd.read_csv(PATH_DATA)

## 概要

`df`は

- 週刊少年ジャンプ
- 週刊少年サンデー
- 週刊少年マガジン
- 週刊少年チャンピオン

の`1970-07-27`から`2017-07-06`までのすべての掲載作品のデータを格納した`DataFrame`です．
まずはサイズを見てみましょう．

In [37]:
df.shape

(179931, 14)

各週の掲載作品を一行ずつ格納しているため，合計で約18万行程度の規模になります．
以下に，各列の構成を示します．

In [38]:
df.columns

Index(['mcname', 'miname', 'cname', 'epname', 'creator', 'pageStart',
       'pageEnd', 'numberOfPages', 'datePublished', 'price', 'publisher',
       'editor', 'pages', 'pageEndMax'],
      dtype='object')

- `mcname`: 雑誌名（**M**gazine **C**ollection **NAME**）
- `miname`: 雑誌巻号名（**M**agazine **I**tem **NAME**）
- `cname`: マンガ作品名（**C**omic **NAME**）
- `epname`: 各話タイトル（**EP**isode **NAME**）
- `creator`: 作者名
- `pageStart`: 開始ページ
- `pageEnd`: 終了ページ
- `numberOfPages`: 雑誌の合計ページ数
- `datePublished`: 雑誌の発行日
- `price`: 雑誌の価格
- `publisher`: 雑誌の出版社
- `editor`: 雑誌の編集者（編集長）
- `pages`: 各話のページ数（`pageEnd` - `pageStart`）
- `pageEndMax`: 雑誌に掲載されているマンガ作品のうち，`pageEnd`の最大値

冒頭数行を見て，データのイメージを掴んでみましょう．

In [40]:
df.head()

,mcname,miname,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor,pages,pageEndMax
0,週刊少年ジャンプ,週刊少年ジャンプ 1970年 表示号数31,男一匹ガキ大将,土佐の源蔵の巻,本宮ひろ志,7.0,37.0,280.0,1970-07-27,80.0,集英社,長野規,30.0,275.0
1,週刊少年チャンピオン,週刊少年チャンピオン 1970年 表示号数14,朝日の恋人,NaN,かざま鋭二,15.0,43.0,292.0,1970-07-27,80.0,秋田書店 ∥ アキタショテン,成田清美,28.0,290.0
2,週刊少年ジャンプ,週刊少年ジャンプ 1970年 表示号数31,ど根性ガエル,男はつらいよの巻,吉沢やすみ,39.0,53.0,280.0,1970-07-27,80.0,集英社,長野規,14.0,275.0
3,週刊少年チャンピオン,週刊少年チャンピオン 1970年 表示号数14,あばしり一家,アバシリ吉三の美人地獄編,永井豪,48.0,66.0,292.0,1970-07-27,80.0,秋田書店 ∥ アキタショテン,成田清美,18.0,290.0
4,週刊少年ジャンプ,週刊少年ジャンプ 1970年 表示号数31,あらし!三匹,ミヒル登場の巻,池沢さとし,56.0,70.0,280.0,1970-07-27,80.0,集英社,長野規,14.0,275.0


`pandas`の`describe()`コマンドでざっくり集計してみます．

In [41]:
df.describe()

,pageStart,pageEnd,numberOfPages,price,pages,pageEndMax
count,179931.000000,179931.000000,179624.000000,179893.000000,179931.000000,179931.000000
mean,210.845105,228.371240,417.376325,203.871740,17.526135,408.249379
std,123.860878,122.038708,67.418723,41.955489,7.712730,69.804101
min,1.000000,1.000000,36.000000,80.000000,0.000000,200.000000
25%,107.000000,126.000000,356.000000,180.000000,16.000000,346.000000
50%,205.000000,222.000000,437.000000,210.000000,18.000000,433.000000
75%,305.000000,322.000000,464.000000,236.000000,19.000000,457.000000
max,581.000000,600.000000,600.000000,371.000000,486.000000,600.000000


In [42]:
df.isna().sum().reset_index()

,index,0
0,mcname,0
1,miname,0
2,cname,9
3,epname,26807
4,creator,441
5,pageStart,0
6,pageEnd,0
7,numberOfPages,307
8,datePublished,0
9,price,38


```{admonition} 前処理が原因の欠測
`numberOfPages`，`price`に関しては，想定外のパターンのデータを`NaN`に変換するよう[前処理](https://kakeami.github.io/viz-madb/appendix/preprocess.html#cm102)を施しています．詳細はAppendixをご参照ください．
```

特に`epname`と`publisher`の欠測が多いことがわかります．

## 列ごとの基礎分析

### `mcname`（雑誌名）

In [43]:
df['mcname'].value_counts().reset_index()

,index,mcname
0,週刊少年サンデー,46172
1,週刊少年チャンピオン,45420
2,週刊少年マガジン,45312
3,週刊少年ジャンプ,43027


上記は`mcname`ごとの行数を表します．それぞれ同一期間で集計しましたが，掲載作品数の違いが生じています．

### `miname`（雑誌巻号名）

ユニークな`miname`数を集計します．

In [44]:
df['miname'].nunique()

9220

`mcname`（雑誌）ごとに集計した`miname`（雑誌巻号）は：

In [45]:
df.groupby('mcname')['miname'].nunique().reset_index()

,mcname,miname
0,週刊少年サンデー,2304
1,週刊少年ジャンプ,2301
2,週刊少年チャンピオン,2314
3,週刊少年マガジン,2301


ほぼ同数ですが，雑誌によって微妙に巻号数が異なることがわかります．

次は，`miname`（雑誌巻号）ごとに`mcname`（マンガ作品）数を集計してみます．

In [46]:
df.groupby('miname')['cname'].nunique().sort_values().reset_index()

,miname,cname
0,週刊少年マガジン 1975年 表示号数1,8
1,週刊少年マガジン 1971年 表示号数1,8
2,週刊少年マガジン 1970年 表示号数51,9
3,週刊少年マガジン 1974年 表示号数45,9
4,週刊少年マガジン 1974年 表示号数44,9
...,...,...
9215,週刊少年マガジン 2010年 表示号数1,35
9216,週刊少年チャンピオン 2004年 表示号数1,37
9217,週刊少年ジャンプ 2014年 表示号数4,37
9218,週刊少年ジャンプ 2003年 表示号数37,40


1970年付近の黎明期は掲載数が少なめだったようです．また，雑誌によってタイミングはバラバラですが，非常に沢山のマンガを掲載する**外れ値**が存在することもわかります．

### `cname`（マンガ作品名）

ユニークな`cname`（マンガ作品）数を集計します．

In [47]:
df['cname'].nunique()

6836

`mcname`（雑誌）ごとに`cname`（マンガ作品）数を集計します．

In [48]:
df.groupby('mcname')['cname'].nunique().reset_index()

,mcname,cname
0,週刊少年サンデー,1486
1,週刊少年ジャンプ,2107
2,週刊少年チャンピオン,1713
3,週刊少年マガジン,1561


ジャンプが圧倒的に多いですね…．試しに`cname`（マンガ作品）ごとに掲載数を集計してみます．

In [49]:
df_tmp = df[['mcname', 'cname']].value_counts().reset_index()
df_tmp.columns = ['mcname', 'cname', 'counts']
df_tmp

,mcname,cname,counts
0,週刊少年ジャンプ,こちら葛飾区亀有公園前派出所,1968
1,週刊少年マガジン,はじめの一歩,1184
2,週刊少年サンデー,名探偵コナン,1009
3,週刊少年ジャンプ,ONE PIECE,890
4,週刊少年サンデー,MAJOR,748
...,...,...,...
6862,週刊少年ジャンプ,大山岳,1
6863,週刊少年ジャンプ,大宮ジェット,1
6864,週刊少年ジャンプ,大喜利大会!!,1
6865,週刊少年ジャンプ,大仮面,1


```{admonition} あれ，あの長編作品は…？
シーズンごとに作品名が変わっているシリーズ作品（ドカベン，刃牙，浦鉄，ジョジョ等）は，それぞれ別作品として集計されていることにご注意ください．
```

連載期間が長いものを見てみましょう．こち亀，はじめの一歩，名探偵コナン，ONEPIECE，MAJORと各雑誌のレジェンドが連なります．
一方で，連載期間が短いものの中には，企画ものや読み切りが存在するようです．

雑誌ごとに，マンガ作品の連載期間に関して基礎集計します．

In [50]:
df_tmp.groupby('mcname')['counts'].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,1486.0,31.069314,73.993766,1.0,1.0,2.0,27.0,1009.0
週刊少年ジャンプ,2107.0,20.420503,75.056735,1.0,1.0,1.0,11.0,1968.0
週刊少年チャンピオン,1713.0,26.514302,60.126900,1.0,1.0,3.0,24.0,635.0
週刊少年マガジン,1561.0,29.024984,75.407882,1.0,1.0,2.0,21.0,1184.0


やはりジャンプの平均連載期間が，他誌と比べて短いことがわかります．

### `epname`（各話タイトル）

ユニークな`epname`（各話タイトル）数を集計します．

In [51]:
df['epname'].nunique()

150310

意外と重複しているようです．集計してみます．

In [52]:
df['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,後編,134
2,前編,118
3,最終回,82
4,第三部 孤狼青春編,81
...,...,...
150305,35発目★ぶりちん 笑え!!笑え!!笑え!!,1
150306,◆第18話◆しゃがらの剣,1
150307,BOX22 / 苦悩と元気の素!!,1
150308,第167話❤恋文,1


プロ野球編ってことは…．

In [53]:
df[df['epname']=='プロ野球編']['cname'].value_counts().reset_index()

,index,cname
0,ドカベン,186


やっぱりドカベンですね！ドカベンってもしかして**〇〇編**の粒度でしかタイトルをつけないのでしょうか…？ドカベンの`epname`を集計してみます．

In [54]:
df[df['cname']=='ドカベン']['epname'].value_counts().reset_index()

,index,epname
0,プロ野球編,186
1,新潟明訓対神奈川明訓,1
2,外伝 不知火・土門伝説,1
3,殿馬一人外伝 秘打「白鳥の湖」誕生秘話,1


In [55]:
df[df['cname']=='ドカベン']['epname'].isna().sum()

446

プロ野球編以外のドカベンの各話タイトルは欠測しているため，これ以上のことはわからなそうです．

### `creator`（作者）

`df`に存在する`creator`（作者）数を集計します．

In [56]:
df['creator'].nunique()

2743

合計作品数が多い`creator`を調べてみます．

In [57]:
df['creator'].value_counts().reset_index().head(10)

,index,creator
0,水島新司,2969
1,秋本治,1876
2,高橋留美子,1736
3,浜岡賢次,1381
4,青山剛昌,1286
5,あだち充,1270
6,森川ジョージ,1224
7,満田拓也,1223
8,板垣恵介,1170
9,藤田和日郎,1102


メンツが強すぎます…．
個人的にはこち亀の`秋本治`先生が一番かなと予想していましたが，`水島新司`先生が圧倒的でした．

ちなみに，こち亀の`creator`を集計すると以下のようになります．

In [58]:
df[df['cname']=='こちら葛飾区亀有公園前派出所']['creator'].value_counts().reset_index()

,index,creator
0,秋本治,1866
1,山止たつひこ,101
2,秋本治ととみさわちなつ,1


`秋本治`先生は，デビュー時`山止たつひこ`というペンネームを使っていました．

（この101話分を足しても全然追いつかない`水島新司`先生がすごすぎますが…）

### `pageStart`（開始ページ）

`pageStart`（開始ページ）について`describe()`で基礎集計すると，以下のようになります．

In [59]:
df['pageStart'].describe().reset_index()

,index,pageStart
0,count,179931.000000
1,mean,210.845105
2,std,123.860878
3,min,1.000000
4,25%,107.000000
5,50%,205.000000
6,75%,305.000000
7,max,581.000000


### `pageEnd`（終了ページ）

`pageEnd`（終了ページ）について`describe()`で基礎集計すると，以下のようになります．

In [60]:
df['pageEnd'].describe().reset_index()

,index,pageEnd
0,count,179931.000000
1,mean,228.371240
2,std,122.038708
3,min,1.000000
4,25%,126.000000
5,50%,222.000000
6,75%,322.000000
7,max,600.000000


### `numberOfPages`（各号の合計ページ数）

`numberOfPages`（雑誌の合計ページ数）を`describe`で基礎集計します．
なお，`df`をそのまま`describe`してしまうと掲載作品数が多い雑誌巻号にバイアスのかかった統計量になってしまうため注意が必要です．
そこで，ここでは`miname`で中間集計した`df_tmp`を`describe`します．

In [79]:
df_tmp = df.groupby('miname')[
    ['numberOfPages']].first().reset_index()
df_tmp['numberOfPages'].describe().reset_index()

,index,numberOfPages
0,count,9207.000000
1,mean,402.711089
2,std,72.049837
3,min,36.000000
4,25%,344.000000
5,50%,420.000000
6,75%,458.000000
7,max,600.000000


最小値が小さすぎる気がします．
試しに`numberOfPages`でソートすると，

In [80]:
df_tmp.sort_values('numberOfPages').head(10)

,miname,numberOfPages
7843,週刊少年マガジン 1989年 表示号数10,36.0
6669,週刊少年チャンピオン 2012年 表示号数28,36.0
5839,週刊少年チャンピオン 1995年 表示号数13,214.0
2480,週刊少年ジャンプ 1974年 表示号数11,234.0
7099,週刊少年マガジン 1974年 表示号数16,236.0
7127,週刊少年マガジン 1974年 表示号数42,236.0
7096,週刊少年マガジン 1974年 表示号数13,236.0
7094,週刊少年マガジン 1974年 表示号数11,236.0
2488,週刊少年ジャンプ 1974年 表示号数19,236.0
2487,週刊少年ジャンプ 1974年 表示号数18,236.0


最初の二つに関しては入力ミスが疑われます．降順にソートしてみます．

In [81]:
df_tmp.sort_values('numberOfPages', ascending=False).head(10)

,miname,numberOfPages
2109,週刊少年サンデー 2013年 表示号数36,600.0
8625,週刊少年マガジン 2005年 表示号数2,574.0
8580,週刊少年マガジン 2004年 表示号数22,573.0
8644,週刊少年マガジン 2005年 表示号数4,572.0
3981,週刊少年ジャンプ 2004年 表示号数37,572.0
8414,週刊少年マガジン 2000年 表示号数49,570.0
1872,週刊少年サンデー 2008年 表示号数4,568.0
2108,週刊少年サンデー 2013年 表示号数35,568.0
2107,週刊少年サンデー 2013年 表示号数34,566.0
1853,週刊少年サンデー 2008年 表示号数2,565.0


特別号の可能性があるので，妥当性の判断が難しいです．

いずれにしても`numberOfPages`は欠測数が多いため積極的に分析に利用せず，後述する`pageEndMax`で代用します．

In [82]:
df_tmp['numberOfPages'].isna().sum()

13

### `datePublished`（発行日）

`datePublished`（発行日）を`describe`で基礎集計します．
前述したように`df`を直接`describe`するとバイアスが乗るので，`miname`で中間集計した`df_tmp`に対して分析を実施します．

In [83]:
df_tmp = df.groupby('miname')[['datePublished']].\
    first().reset_index()
# 日付処理を容易にするため，`pd.to_datetime`で型変換
df_tmp['datePublished'] = pd.to_datetime(df_tmp['datePublished'])
df_tmp['datePublished'].describe().reset_index()

,index,datePublished
0,count,9220
1,unique,6365
2,top,1983-01-01 00:00:00
3,freq,4
4,first,1970-07-27 00:00:00
5,last,2017-07-12 00:00:00


次に，年単位で集計してみます．

In [84]:
df_tmp['yearPublished'] = df_tmp['datePublished'].dt.year
df_tmp.value_counts('yearPublished').reset_index().\
    sort_values('yearPublished', ignore_index=True)

,yearPublished,0
0,1970,86
1,1971,204
2,1972,204
3,1973,204
4,1974,199
5,1975,202
6,1976,200
7,1977,202
8,1978,202
9,1979,200


集計開始年（`1970`）および集計終了年（`2017`）以外は，年間およそ190-205回ほど発行していることがわかります．

### `price`（雑誌価格）

`price`（雑誌価格）を`describe`で基礎集計します．
前述したように`df`を直接`describe`するとバイアスが乗るので，`miname`で中間集計した`df_tmp`に対して分析を実施します．

In [85]:
df_tmp = df.groupby('miname')[['price']].\
    first().reset_index()
df_tmp['price'].describe().reset_index()

,index,price
0,count,9218.000000
1,mean,195.055544
2,std,44.799162
3,min,80.000000
4,25%,170.000000
5,50%,200.000000
6,75%,229.000000
7,max,371.000000


一冊80円だった時代があったのでしょうか…？後ほど分析します．

### `publisher`（出版社）

`publisher`（出版社）に関して集計します． 前述したように`df`を直接集計するとバイアスが乗るので，minameで中間集計したdf_tmpに対して分析を実施します．

In [86]:
df_tmp = df.groupby('miname')[['mcname', 'publisher']].\
    first().reset_index()

雑誌ごとに出版社名を集計します．

In [87]:
df_tmp.groupby('mcname')['publisher'].\
    value_counts().reset_index(name='count')

,mcname,publisher,count
0,週刊少年サンデー,小学館 ∥ ショウガクカン,1715
1,週刊少年サンデー,小学館,222
2,週刊少年サンデー,"小学館 ∥ ショウガクカン,小学館",55
3,週刊少年ジャンプ,集英社 ∥ シュウエイシャ,1335
4,週刊少年ジャンプ,"集英社 ∥ シュウエイシャ,集英社",383
5,週刊少年ジャンプ,集英社,378
6,週刊少年チャンピオン,秋田書店 ∥ アキタショテン,1623
7,週刊少年チャンピオン,秋田書店 ∥ アキタ ショテン,338
8,週刊少年チャンピオン,秋田書店,56
9,週刊少年チャンピオン,"秋田書店 ∥ アキタショテン,秋田書店 ∥ アキタ ショテン",1


かなり表記がぶれているようですが，今後積極的に使う情報ではないため，このままにしておきます．

### `editor`（編集者）

`editor`（編集者）に関して集計します． 前述したように`df`を直接集計するとバイアスが乗るので，`miname`で中間集計した`df_tmp`に対して分析を実施します．

In [88]:
df_tmp = df.groupby('miname')[['mcname', 'editor']].\
    first().reset_index()

In [89]:
df_tmp.groupby('mcname')['editor'].value_counts().\
    reset_index(name='count')

,mcname,editor,count
0,週刊少年サンデー,田中一喜,384
1,週刊少年サンデー,熊谷玄典,301
2,週刊少年サンデー,井上敬三,244
3,週刊少年サンデー,林正人,226
4,週刊少年サンデー,奥山豊彦,176
5,週刊少年サンデー,三上信一,156
6,週刊少年サンデー,猪俣光一郎,148
7,週刊少年サンデー,平山隆,145
8,週刊少年サンデー,縄田正樹,142
9,週刊少年サンデー,鳥光裕,127


誤記と思われるものがいくつかあります：

- `週刊少年サンデー`
    - `三上伸一`さん（おそらく`三上信一`さん？）
- `週刊少年ジャンプ`
    - `鳥島和彦`さん（おそらく`鳥嶋和彦`さん？）
    - `高校俊昌`さん（おそらく`高橋俊昌`さん？）

数も少ないですし，確証が持てないため特に修正しないことにします．

### `pages`（各話のページ数）

`pages`（各話のページ数）を`describe`で基礎集計します．

In [75]:
df['pages'].describe().reset_index()

,index,pages
0,count,179931.000000
1,mean,17.526135
2,std,7.712730
3,min,0.000000
4,25%,16.000000
5,50%,18.000000
6,75%,19.000000
7,max,486.000000


四分位は納得感がありますが，最小・最大が想定外です．昇順で並び替えてみます．

In [76]:
df.sort_values('pages').head()

,mcname,miname,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor,pages,pageEndMax
16949,週刊少年ジャンプ,週刊少年ジャンプ 1976年 表示号数48,ミオとヒロシの週間パトロール,文化祭シーズン ／ トンガ力士廃業,斉藤ヒロシ,315.0,315.0,382.0,1976-11-29,170.0,集英社,中野祐介,0.0,375.0
4908,週刊少年サンデー,週刊少年サンデー 1972年 表示号数20,ウルトラ・ナンセンス劇場!,ええかげんなエレベーターガール,辰野落児,202.0,202.0,286.0,1972-05-07,90.0,小学館 ∥ ショウガクカン,井上敬三,0.0,283.0
12588,週刊少年ジャンプ,週刊少年ジャンプ 1975年 表示号数16,鹿兵のケラケラ日記,どれがホンモノ?都民党 ／ 野球,南田鹿兵,244.0,244.0,268.0,1975-04-21,130.0,集英社 ∥ シュウエイシャ,中野祐介,0.0,262.0
109575,週刊少年チャンピオン,週刊少年チャンピオン 2002年 表示号数12,でじこの1ページ劇場,第42幕,コゲどんぼ,430.0,430.0,437.0,2002-02-28,210.0,秋田書店 ∥ アキタ ショテン,樋口茂,0.0,436.0
118863,週刊少年ジャンプ,週刊少年ジャンプ 2004年 表示号数20,BLEACH,-17. 逸れゆく星々の為の前奏曲,久保帯人,2.5,2.5,416.0,2004-04-26,219.0,集英社 ∥ シュウエイシャ,茨木政彦,0.0,413.0


`週刊少年ジャンプ 2004年 表示号数20`について[メディア芸術データベースで調べた](https://mediaarts-db.bunka.go.jp/id/M543111/)ところ，`BLEACH`は通常連載に加えて綴込み付録で`-17. 逸れゆく星々の為の前奏曲`を掲載したようです．
`pageStart`および`pageEnd`が中途半端な小数になっているのはそのためでしょう．

In [77]:
df.sort_values('pages', ascending=False).head()

,mcname,miname,cname,epname,creator,pageStart,pageEnd,numberOfPages,datePublished,price,publisher,editor,pages,pageEndMax
172306,週刊少年ジャンプ,週刊少年ジャンプ 2015年 表示号数48,磯部磯兵衛物語～浮世はつらいよ～,第161話 御目出度いのに!で候,仲間りょう,3.0,489.0,498.0,2015-11-09,255.0,NaN,瓶子吉久,486.0,489.0
92463,週刊少年サンデー,週刊少年サンデー 1998年 表示号数14,アクシデンツ,CASE83.ナギの島2,山田貴敏,7.0,461.0,474.0,1998-03-18,230.0,小学館 ∥ ショウガクカン,奥山豊彦,454.0,461.0
76857,週刊少年マガジン,週刊少年マガジン 1994年 表示号数24,宮田すすむと日本の社長!,社長登場! ／ 面接 ／ サイン,愛本ゆみ,3.0,394.0,428.0,1994-06-01,200.0,講談社 ∥ コウダンシャ,五十嵐隆夫,391.0,417.0
171023,週刊少年サンデー,週刊少年サンデー 2015年 表示号数35,アンペア,第9話 侵入者,山仲剛太,283.0,600.0,506.0,2015-07-29,270.0,NaN,市原武法,317.0,600.0
111687,週刊少年マガジン,週刊少年マガジン 2002年 表示号数36,RAVE,RAVE:152 裸の英雄!?,真島ヒロ,39.0,272.0,508.0,2002-08-21,229.0,"講談社 ∥ コウダンシャ,講談社",野内雅宏,233.0,495.0


`週刊少年ジャンプ 2015年 表示号数48`について[メディア芸術データベースで調べた](https://mediaarts-db.bunka.go.jp/id/M675741/)ところ，`磯部磯兵衛物語～浮世はつらいよ～`は二周年記念で巻頭カラーをもらっていたようです．巻末の定位置での掲載はなかったことから，おそらく`pageEnd`は誤入力と思われます．

### `pageEndMax`（雑誌巻号の最大の`pageEnd`）

`pageEndMax`（雑誌巻号の最大の`pageEnd`）えお`describe`で基礎集計します．なお，`df`をそのまま`describe`してしまうと掲載作品数が多い雑誌巻号にバイアスのかかった統計量になってしまうため注意が必要です． そこで，ここでは`miname`で中間集計した`df_tmp`を`describe`します．

In [78]:
df_tmp = df.groupby('miname')[
    ['pageEndMax']].first().reset_index()
df_tmp['pageEndMax'].describe().reset_index()

,index,pageEndMax
0,count,9220.000000
1,mean,392.943059
2,std,74.678356
3,min,200.000000
4,25%,326.000000
5,50%,410.000000
6,75%,451.000000
7,max,600.000000
